In [1]:
import xarray as xr
import numpy as np
import os
from surgeNN.surgeNN.evaluation import rmse, compute_precision, compute_recall, compute_f1, add_error_metrics_to_prediction_ds

2025-01-06 12:47:56.745665: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-06 12:47:56.807792: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
#configure the script
tgs        = ['stavanger-svg-nor-nhs.csv','wick-wic-gbr-bodc.csv','esbjerg-esb-dnk-dmi.csv','immingham-imm-gbr-bodc.csv','den_helder-denhdr-nld-rws.csv', 'fishguard-fis-gbr-bodc.csv',  'brest-822a-fra-uhslc.csv', 'vigo-vigo-esp-ieo.csv',  'alicante_i_outer_harbour-alio-esp-da_mm.csv']
tgnames = ['Stavanger (NOR)','Wick (UK)', 'Esbjerg (DK)','Immingham (UK)','Den Helder (NL)','Fishguard (UK)','Brest (FR)','Vigo (PT)', 'Alicante (SP)']

qnts = np.array([.95,.98,.99,.995]) #quantiles, don't touch

Update performance output datasets:

1) remove '*_self' outputs
2) add 'max_timesteps_between_extremes' as a dimension for original results (generated without 'max_timesteps_between_extremes'), and add results for max_timesteps_between_extremes = 3

**be careful as this overwrites results!**

In [4]:
#MLR
path = '/home/jovyan/test_surge_models/results/mlr_4x4/performance'
files = os.listdir(path)
for f in files:
    if f.startswith('.') == False:
        print('current_file:' +f)
        ds = xr.open_dataset(os.path.join(path,f)).load()
        
        #1
        ds = ds.drop_vars(['true_pos_self','true_neg_self','false_pos_self','false_neg_self',
                            'precision_self','recall_self','f1_self'],errors='ignore')
        
        #2
        if 'max_timesteps_between_extremes' in ds:
            print('"max_timesteps_between_extremes" is already a dimension, file already updated? Skipping.')
        else:   
            for metric in ['r_extremes','rmse_extremes','true_pos','false_neg','false_pos','true_neg','precision','recall','f1']:
                ds[metric] = ds[metric].expand_dims(dim='max_timesteps_between_extremes') #add additional dimension for max time distance used between 

            ds['max_timesteps_between_extremes'] = [-1]

            ds_max_dist_3 = add_error_metrics_to_prediction_ds(ds.copy(deep=True),qnts,3)
            ds_new = xr.merge((ds,ds_max_dist_3))

            ds.close()
            ds_new.to_netcdf(os.path.join(path,f.replace('.nc','_v2.nc')),mode='w') #store new dataset
            del ds,ds_new
            
            os.remove(os.path.join(path,f)) #delete old file
            os.rename(os.path.join(path,f.replace('.nc','_v2.nc')),os.path.join(path,f))#rename new file

current_file:mlr_4x4_3h_esbjerg-esb-dnk-dmi.nc
current_file:mlr_4x4_3h_wick-wic-gbr-bodc.nc
current_file:mlr_4x4_3h_den_helder-denhdr-nld-rws.nc
current_file:mlr_4x4_3h_immingham-imm-gbr-bodc.nc
current_file:mlr_4x4_3h_brest-822a-fra-uhslc.nc
current_file:mlr_4x4_3h_stavanger-svg-nor-nhs.nc
current_file:mlr_4x4_3h_alicante_i_outer_harbour-alio-esp-da_mm.nc
current_file:mlr_4x4_3h_vigo-vigo-esp-ieo.nc
current_file:mlr_4x4_3h_fishguard-fis-gbr-bodc.nc


In [5]:
#LSTMs
path = '/home/jovyan/test_surge_models/results/nns/performance/lstm'
files = os.listdir(path)
for f in files:
    if f.startswith('.') == False:
        print('current_file:' +f)
        ds = xr.open_dataset(os.path.join(path,f))
        
        #1
        ds = ds.drop_vars(['true_pos_self','true_neg_self','false_pos_self','false_neg_self',
                            'precision_self','recall_self','f1_self'],errors='ignore')
        
        #2
        if 'max_timesteps_between_extremes' in ds:
            print('"max_timesteps_between_extremes" is already a dimension, file already updated? Skipping.')
        else:
            for metric in ['r_extremes','rmse_extremes','true_pos','false_neg','false_pos','true_neg','precision','recall','f1']:
                ds[metric] = ds[metric].expand_dims(dim='max_timesteps_between_extremes') #add additional dimension for max time distance used between 

            ds['max_timesteps_between_extremes'] = [-1]

            ds_max_dist_3 = add_error_metrics_to_prediction_ds(ds.copy(deep=True),qnts,3)
            ds_new = xr.merge((ds,ds_max_dist_3))

            ds.close()
            ds_new.to_netcdf(os.path.join(path,f.replace('.nc','_v2.nc')),mode='w') #store new dataset
            del ds,ds_new

            os.remove(os.path.join(path,f)) #delete old file
            os.rename(os.path.join(path,f.replace('.nc','_v2.nc')),os.path.join(path,f))#rename new file

current_file:lstm_3h_brest-822a-fra-uhslc_mse_hp1_it0.nc
current_file:lstm_3h_den_helder-denhdr-nld-rws_mse_hp1_it2.nc
current_file:lstm_3h_wick-wic-gbr-bodc_mse_hp1_it4.nc
current_file:lstm_3h_brest-822a-fra-uhslc_mse_hp1_it3.nc
current_file:lstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it1.nc
current_file:lstm_3h_esbjerg-esb-dnk-dmi_mse_hp1_it2.nc
current_file:lstm_3h_wick-wic-gbr-bodc_mse_hp1_it1.nc
current_file:lstm_3h_stavanger-svg-nor-nhs_mse_hp1_it2.nc
current_file:lstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it3.nc
current_file:lstm_3h_wick-wic-gbr-bodc_mse_hp1_it3.nc
current_file:lstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it4.nc
current_file:lstm_3h_immingham-imm-gbr-bodc_mse_hp1_it0.nc
current_file:lstm_3h_esbjerg-esb-dnk-dmi_mse_hp1_it0.nc
current_file:lstm_3h_stavanger-svg-nor-nhs_mse_hp1_it4.nc
current_file:lstm_3h_vigo-vigo-esp-ieo_mse_hp1_it2.nc
current_file:lstm_3h_immingham-imm-gbr-bodc_mse_hp1_it2.nc
current_file:lstm_3h_vigo-vigo-esp

In [6]:
#ConvLSTMs
path = '/home/jovyan/test_surge_models/results/nns/performance/convlstm'
files = os.listdir(path)
for f in files:
    if f.startswith('.') == False:
        print('current_file:' +f)
        ds = xr.open_dataset(os.path.join(path,f))
        
        #1
        ds = ds.drop_vars(['true_pos_self','true_neg_self','false_pos_self','false_neg_self',
                            'precision_self','recall_self','f1_self'],errors='ignore')
        
        #2
        if 'max_timesteps_between_extremes' in ds:
            print('"max_timesteps_between_extremes" is already a dimension, file already updated? Skipping.')
        else:
            for metric in ['r_extremes','rmse_extremes','true_pos','false_neg','false_pos','true_neg','precision','recall','f1']:
                ds[metric] = ds[metric].expand_dims(dim='max_timesteps_between_extremes') #add additional dimension for max time distance used between 

            ds['max_timesteps_between_extremes'] = [-1]

            ds_max_dist_3 = add_error_metrics_to_prediction_ds(ds.copy(deep=True),qnts,3)
            ds_new = xr.merge((ds,ds_max_dist_3))

            ds.close()
            ds_new.to_netcdf(os.path.join(path,f.replace('.nc','_v2.nc')),mode='w') #store new dataset
            del ds,ds_new

            os.remove(os.path.join(path,f)) #delete old file
            os.rename(os.path.join(path,f.replace('.nc','_v2.nc')),os.path.join(path,f))#rename new file

current_file:convlstm_3h_immingham-imm-gbr-bodc_mse_hp1_it2.nc
current_file:convlstm_3h_esbjerg-esb-dnk-dmi_mse_hp1_it2.nc
current_file:convlstm_3h_fishguard-fis-gbr-bodc_mse_hp1_ndeg1.5_it4.nc
current_file:convlstm_3h_esbjerg-esb-dnk-dmi_mse_hp1_ndeg1.5_it3.nc
current_file:convlstm_3h_den_helder-denhdr-nld-rws_mse_hp1_it0.nc
current_file:convlstm_3h_vigo-vigo-esp-ieo_mse_hp1_it1.nc
current_file:convlstm_3h_stavanger-svg-nor-nhs_mse_hp1_it3.nc
current_file:convlstm_3h_den_helder-denhdr-nld-rws_mse_hp1_it1.nc
current_file:convlstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it4.nc
current_file:convlstm_3h_esbjerg-esb-dnk-dmi_mse_hp1_ndeg1.5_it4.nc
current_file:convlstm_3h_fishguard-fis-gbr-bodc_mse_hp1_ndeg1.5_it1.nc
current_file:convlstm_3h_esbjerg-esb-dnk-dmi_mse_hp1_ndeg1.5_it2.nc
current_file:convlstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it2.nc
current_file:convlstm_3h_alicante_i_outer_harbour-alio-esp-da_mm_mse_hp1_it0.nc
current_file:convlstm_3h_brest-822a-fra